# Functionality of this notebook: 

* calculate the ciCOH for normal and mild LFP data in rest

## Import Packages

In [35]:
import os, sys
import scipy.io as sio
import glob
import numpy as np
import matplotlib.pyplot as plt
import pickle 
import math
%matplotlib inline

In [36]:
# extract the exp folder path
currfolder = os.getcwd()
codefolder = currfolder[0 : currfolder.find('code')+len('code')]

# add path the exp folder
sys.path.append(codefolder)

# import_nbmodule used for import package in .ipynb
import import_nbmodule

# import util/folder_extract.pynb 
from util.folder_extract import exp_subfolders, code_corresfolder

# import ciCoherence_overtime in connAnalyTool/synchronization_indices.ipynb
from connAnalyTool.synchronization_indices import ciCoherence_acrosstrials
from connAnalyTool.synchronization_indices import ciCoherence_overtime

## exp subfolders & code_corresfolder

In [37]:
_, _, pipelinefolder, _= exp_subfolders()

In [38]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [39]:
nb_name = nb_name[0: nb_name.find('.ipynb')]

# corresfolder
corresfolder = code_corresfolder(os.getcwd(), nb_name)

## input setup

In [40]:
inputfolder = os.path.join(pipelinefolder, 'NHP_Pinky', '1_dataAnaly', 'FCAnaly', 'Rest', 'm0_restDataGM_narrowfiltered9_11')

In [41]:
filenames = ['Pinky_narrowFiltered9_11-PreprodRestdataGM_mild_110917_bktdt4.mat',
             'Pinky_narrowFiltered9_11-PreprodRestdataGM_normal_041417_bktdt6.mat',
             'Pinky_narrowFiltered9_11-PreprodRestdataGM_normal_042117_bktdt1.mat']

## global parameters

In [42]:
freq = [9, 11]

# time duration used for calculating ciCOH
ts = np.asarray([60, 60*4]);

# time window duration and the shift duration
t_win, t_shift = 10, 5

## Save Setup

In [43]:
savefolder = corresfolder
savefilename_prefix =  'ciCOH_rest' + '_freq' + str(freq[0]) + '_' + str(freq[1]) + '_'

## load data

### <span style="color:red"> should modified as forloop </span>

In [44]:
filei = 1

In [45]:
filename = filenames[filei]
file = os.path.join(inputfolder, filename)

In [46]:
# load 
matdat = sio.loadmat(file, variable_names = ['lfpdata', 'fs', 'chnsarea_vNames', 'chnsarea_Cell'], 
                     struct_as_record = False, squeeze_me = True)

In [47]:
# fs: sample rate
fs = matdat['fs'] 

# chnsarea information: chnsarea_Cell + chnsarea_vNames = T_chnsarea (matlab table obj)
chnsarea_Cell, chnsarea_vNames = matdat['chnsarea_Cell'], matdat['chnsarea_vNames']

# lfp (np.ndarray): ntemporal * nchns
lfpdata = matdat['lfpdata']

In [48]:
print(chnsarea_Cell)

[array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27], dtype=uint8)
 array(['lSMA', 'lSMA', 'lSMA', 'lSMA', 'rSMA', 'rSMA', 'rSMA', 'rSMA',
       'rMC', 'rMC', 'lVA', 'lVA', 'rVA', 'rVA', 'rVA', 'lVLo', 'lVLo',
       'rVLo', 'rVLo', 'lVPLo', 'lVPLo', 'lVPLo', 'rVPLo', 'rVPLo',
       'rVPLo', 'lCd', 'lCd'], dtype=object)
 array([101, 102, 107, 112, 117, 122, 127, 128, 118, 123, 103, 104, 124,
       130, 131, 110, 115, 119, 120, 106, 111, 116, 121, 126, 132, 108,
       113], dtype=uint8)]


## Calculate ciCOH

### preprocessing: 
select temporal inteval and segment into time-windows

In [25]:
# get the epoch of ts time for using, before and after were discarded to avoid any transient activity
idx_temporal = (ts * fs).astype(int);
lfp = lfpdata[idx_temporal[0]:idx_temporal[1],:]

In [26]:
###  segment to t_win duration time-window
nwintempo, nshift = int(t_win * fs), int(t_shift * fs)
ntempo, nchns = lfp.shape
# nwins: total number of segmented time-window
nwins = math.floor((ntempo - nwintempo + 1) / nshift)

lfpwins = np.empty([nwintempo, nchns, nwins])
for wini in range(nwins):
    idx_start = wini * nshift
    idx_end = idx_start + nwintempo
    lfpwins[:,:,wini] = lfp[idx_start:idx_end, :]

### calculate ciCOH for all time-window

In [27]:
### calculate ciCOH
ntempo, nchns, nwins = lfpwins.shape
ciCOH_allWins = np.zeros((nchns, nchns, nwins))
for wini in range(nwins):
    for chni in range(nchns -1):
        signal1 = lfpwins[:, chni, wini]
        
        for chnj in range(chni+1, nchns):
            signal2 = lfpwins[:, chnj, wini]
            
            # ciCOH_allWins assignment
            ciCOH_allWins[chni, chnj, wini] = ciCoherence_overtime(signal1, signal2)
            
            # symmetrical
            ciCOH_allWins[chnj, chni, wini] = ciCOH_allWins[chni, chnj, wini]
            
            del signal2
        del signal1

In [28]:
ciCOH = np.mean(ciCOH_allWins, axis = 2)

## save ciCOH

In [33]:
if 'normal' in filename:
    idx = filename.find('normal')
if 'mild' in filename:
    idx = filename.find('mild')
savefile = os.path.join(savefolder, savefilename_prefix + filename[idx:])

In [34]:
## save ciCOH
np.save (savefile, ciCOH)